# "Recon2.2plusthreeCompetition" model loading

In [1]:
import cobra
import pandas as pd
import numpy as np
import pickle
from cobra.flux_analysis import pfba

In [2]:
from cobra.flux_analysis import pfba
# import map visualization tools
from utils import show_map
import escher
map_loc = 'Recon2.2plusthreeCompetitiveIIforObesityMap.json' # the escher map used below
# show all output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Load Recon2.2Model with a simplified medium for the course
M = cobra.io.load_json_model('Recon2.2plusthreeCompetitive.json')

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


In [3]:
model=M.copy()

In [4]:
# Setting objective function: biomass reaction in the brain + dopamine production + serotonin production
quadratic_objective = model.problem.Objective(model.reactions.biomass_reaction_INB.flux_expression+
                                              model.reactions.get_by_id('3HLYTCL_INB').flux_expression+
                                              model.reactions.get_by_id('5HLTDL_INB').flux_expression,
    direction='max')
model.objective = quadratic_objective

In [5]:
# Changing the phe and tyr in the medium, because they also use for producing dopa.
model.reactions.EX_phe_L_e.lower_bound=-0.2
model.reactions.EX_phe_L_e.upper_bound=-0.2
model.reactions.EX_tyr_L_e.lower_bound=-0.1

In [6]:
## Other reactions to produce l-dopa is forbidded
model.reactions.TYRDOPO_INB.upper_bound=0
model.reactions.TYR3MO2_INB.upper_bound=0
## liver biomass is constant
model.reactions.biomass_reaction.lower_bound=0.05
model.reactions.biomass_reaction.upper_bound=0.05
## do not need too much biomass in the brain
model.reactions.biomass_reaction_INB.upper_bound=0.1
# Phe secretion in the blood is blocked, otherwise there is no competition.
model.reactions.EX_phe_L_BD.upper_bound=0
# Phenylpyruvate production in the brain and Liver are limited
model.reactions.get_by_id('PHPYRte_INB').lower_bound=0
model.reactions.get_by_id('PHPYRte').lower_bound=0
# Serotonin production
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.095
### No PAH in the brain
model.reactions.PHETHPTOX2_INB.lower_bound=0
model.reactions.PHETHPTOX2_INB.upper_bound=0
model.reactions.r0399_INB.lower_bound=0
model.reactions.r0399_INB.upper_bound=0
### Tyrosine hydroxylase
# model.reactions.EX_TH_B_INB.upper_bound=1000
model.reactions.get_by_id('34dhpheTR_TOBD').upper_bound=0 ##No levodopa from liver to blood.
model.reactions.Ti1_To1.upper_bound=0.198
model.reactions.Phe_TH_INB.upper_bound=0
model.reactions.PhePde_INB.upper_bound=0
model.solver='glpk'
sol = model.optimize()
sol.objective_value
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

0.2547554

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [7]:
### Essential need for model
model.reactions.biomass_reaction_INB.lower_bound=0.1
model.reactions.get_by_id('3HLYTCL_INB').lower_bound=0.0567
model.reactions.get_by_id('5HLTDL_INB').lower_bound=0.098

In [10]:
# exchanges = [ rxn for rxn in model.reactions if rxn.products == [] and 'EX_' in rxn.id ]
# for rxn in exchanges:
#     rxn.upper_bound = 999999

In [9]:
model.reactions.EX_tag_hs_e.upper_bound=1000

In [10]:
model.objective='EX_tag_hs_e'
model.solver='glpk'
sol = model.optimize()
sol.objective_value

0.046584036736021066

In [11]:
fbasol = model.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [13]:
model1=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model1.genes.get_by_id('HGNC:8582').knock_out()
model.reactions.get_by_id('PHPYRte_INB').lower_bound=-0.1
model.reactions.get_by_id('PHPYRte').lower_bound=-0.1
model1.reactions.EX_phe_L_e.upper_bound=-0.04
model1.reactions.EX_phe_L_e.lower_bound=-0.04
## The Phe-restricted diet contains more carbohydrate content, We assumed more glucose.
model1.reactions.EX_glc_e.lower_bound=-2
sol = model1.optimize()
sol.objective_value

0.11536840700733109

In [14]:
fbasol = model1.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [15]:
## Normal person uptake more glucose
model2=model.copy()
model2.reactions.EX_glc_e.lower_bound=-2
sol = model2.optimize()
sol.objective_value

0.12967034081500725

In [16]:
fbasol = model2.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…

In [17]:
## PKU patient uptake same glucose as normal person
model3=model.copy()
# Kncoking gene 'HGNC:8582' to simulate the PKU patients
model3.genes.get_by_id('HGNC:8582').knock_out()
model3.reactions.EX_phe_L_e.upper_bound=-0.04
model3.reactions.EX_phe_L_e.lower_bound=-0.04
## The Phe-restricted diet contains more carbohydrate content, We assumed more glucose.
model3.reactions.EX_glc_e.lower_bound=-1
sol = model3.optimize()
sol.objective_value

0.032574099115998435

In [18]:
fbasol = model3.optimize()
b = show_map(fbasol,map_loc)
b

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'13DAMPPOX': 0.0, '1a_24_25VI…